# Import thư viện

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt
from imutils import perspective
import torch
import os

In [ ]:
!rm -rf /content/drive/MyDrive/cv/runs

In [ ]:
%cd /content/drive/MyDrive/cv
!pip install ultralytics

/content/drive/MyDrive/cv
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 695.2/695.2 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="4CVrVFFaJskPhfMK9n2w")
project = rf.workspace("hamac03").project("-license-plate-detection")
dataset = project.version(1).download("yolov8")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6
  Attempting uninstall: cycler
    Found existing installation: cycle

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.238, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to -license-plate-detection--1 in yolov8:: 100%|██████████| 3508/3508 [00:18<00:00, 193.71it/s]


In [ ]:
from ultralytics import YOLO

# Training YOLOv8n

In [ ]:
!yolo task=detect mode=train model=yolov8n.pt data={'/content/drive/MyDrive/cv/license-plate_detection'}/data.yaml epochs=50 batch=32 imgsz=640

Ultralytics YOLOv8.0.238 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/cv/license-plate_detection/data.yaml, epochs=50, time=None, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_l

# Evaluate

In [ ]:
model = YOLO('/content/drive/MyDrive/cv/runs/detect/train/weights/best.pt')  # load a custom model

# Validate the model
metrics = model.val(split = 'test', batch=1)

Ultralytics YOLOv8.0.238 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/cv/license-plate_detection/test/labels... 349 images, 0 backgrounds, 0 corrupt: 100%|██████████| 349/349 [00:01<00:00, 237.37it/s]


val: New cache created: /content/drive/MyDrive/cv/license-plate_detection/test/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 349/349 [00:09<00:00, 36.52it/s]


                   all        349        349          1          1      0.995      0.941
Speed: 0.8ms preprocess, 14.5ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to runs/detect/val


In [ ]:
print(f'Precision: {metrics.box.p.item():.4f}')
print(f'Recall: {metrics.box.r.item():.4f}')
print(f'F1-score: {metrics.box.r.item():.4f}')
print(f'mAP@.5: {metrics.box.map50.item():.4f}')
print(f'mAP[.5:.95]: {metrics.box.maps.item():.4f}')

Precision: 0.9998
Recall: 1.0000
F1-score: 1.0000
mAP@.5: 0.9950
mAP[.5:.95]: 0.9410
